RAG(검색 증강 생성) 시스템의 핵심인 ChromaDB를 이용한 데이터 관리(CRUD)과정입니다. 이 코드를 "벡터 데이터베이스"와 "임베딩"이라는 두 가지 핵심 개념을 중심.

## 핵심 개념 1: 벡터 데이터베이스 (Vector Database)

일반적인 RDBMS가 정형화된 숫자나 문자열 데이터를 다룬다면, 벡터 데이터베이스는 '의미'를 담고 있는 숫자 배열(벡터)을 저장하고 검색하기 위해 특화된 데이터베이스.

1. 왜 벡터 DB가 필요한가요?

RDBMS (SQL)                            	벡터 DB (ChromaDB)
저장 데이터	                        이름, 가격, 날짜 등 정형화된 값
검색 방법	                        키워드 일치 (WHERE name = '홍길동')
한계	                            키워드가 정확히 일치해야만 검색 가능

ChromaDB는 이러한 의미 기반 검색을 위해 특별히 설계된 DB로, RAG 시스템에서"내가 가진 지식 문서를 저장하는 역할.

2. 컬렉션 (Collection)

*코드 `collection = client.get_or_create_collection(name='test', ...)`
*세부 설명 RDBMS의테이블(Table)과 같습니다. 데이터를 논리적으로 분리하고 정리하는 공간. 이 컬렉션은 특정임베딩 모델(`embedding_fn`을 사용하도록 고정되어, 여기에 저장되는 모든 벡터는 일관된 규칙을 따름.

## 핵심 개념 2: 임베딩 함수 (Embedding Function)

임베딩 함수는텍스트를 벡터로 변하는 역할. 이는 컴퓨터가 단어나 문장의'의미'를 숫자로 인하는 유일한 방법.

1. 트랜스포머와 `all-MiniLM-L6-v2`

*코드 `embedding_fn = SentenceTransformerEmbeddingFunction(model_name='all-MiniLM-L6-v2')`
*세부 설명 이 함수는Sentence-Transformer 라이브러리의MiniL이라는트랜스포머 기반 딥러닝 모을 사용.
    *트랜스포머의 역할 문장의 문맥을 파악하여 각 단어와 문장 전체의 의미를 추출.
    *결과 추출된 의미는384차의 숫자 배열(벡터)로 압축되어 나옴. 이 벡터의 각 숫자가 텍스트의 미묘한 특징(뉘앙스)을 담고 있음.

2. 저장 시 변환 과정 (`collection.add()`)

*코드 `collection.add(documents=texts, embeddings=embeddings, ...)`
*세부 설명 `add` 함수를 호출할 때, ChromaDB는 내부적으로 `texts`를 가져와 `embedding_fn`에 넣고, 반환된384차원 벡(`embeddings`)를원본 텍스트와 함 저장. 이 벡터가 나중에 검색의 기준.

---

## 핵심 기능: CRUD의 역할

실습한 CRUD(생성/읽기/수정/삭제) 과정은 벡터 DB를지식 기반(Knowledge Base)으로 관리하는 데 필수적.

| ChromaDB 함수 | SQL 명령어 | RAG 시스템에서의 역할 |
| :--- | :--- | :--- |
|`collection.add() | `INSERT INTO` |새로운 지식 문서를 벡터화하여 DB에색인(Indexing). |
|`collection.get() | `SELECT *` | 저장된 문서의 원본 내용이나 메타데이가 올바르게 들어갔는지 확인. |
|`collection.update() | `UPDATE` | 지식 문서가 업데이트되었을 때, 기존 ID의 텍스트와 벡터를 최신 내용으로 교체. (이때 새 벡터가 다시 생성됨) |
|`collection.delete() | `DELETE FROM` | 폐기되거나 잘못된 오래된 지식을 DB에서 제거하여 잘못된 정보의 검색을 방지 |

이처럼 ChromaDB는 RDBMS와 유사한 데이터 관리 체계를 가지지만, 최종 목적은"의미"를 검하는 것.

In [1]:
!pip install sentence-transformers chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 1.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.8/20.8 MB 31.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 55.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 21.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.3/132.3 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.9/65.9 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.0/208.0 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 7.0 MB/s eta 

In [ ]:
import chromadb # ChromaDB 라이브러리 임포트
from chromadb import PersistentClient    # 영구 저장을 위한 PersistentClient 임포트 (DB 접속자 역할)
from chromadb.utils.embedding_functions import SentenceTransformerEmbeddingFunction # 임베딩 함수를 명시적으로 정의하기 위한 클래스 임포트


# 1. 임베딩 함수 정의: 텍스트를 벡터로 변환하는 모델 지정
# all-MiniLM-L6-v2 모델 사용 (384차원 벡터 생성)
embedding_fn = SentenceTransformerEmbeddingFunction(model_name='all-MiniLM-L6-v2')

# 2. 클라이언트 연결: 영구 저장소(./chroma_db 폴더)를 설정
client = PersistentClient(path='./chroma_db')

# 3. 컬렉션 생성 (RDBMS의 테이블 생성과 유사)
# get_or_create_collection: 컬렉션이 없으면 생성, 있으면 가져옴
# embedding_function을 명시적으로 지정하여 이 컬렉션의 벡터 규칙을 고정

# client.delete_collection(name='test')
collection = client.get_or_create_collection(name='test', embedding_function=embedding_fn)

# 4. 컬렉션에 자료 추가 (add - SQL의 INSERT)
collection.add(
    documents=[
        "문서1:금요일 퇴근 후에 헬스장",
        "문서2:잠은 언제 자나~"
    ],
    metadatas=[
        {'tag': 'mes1'},    # 부가 정보 (검색 필터링 용도로 사용 가능)
        {'tag': 'mes2'}
    ],
    ids=["doc1", "doc2"]    # 각 문서의 고유 ID (Primary Key 역할)
)

# 5. 자료 조회 (get - SQL의 SELECT)
# include=['documents','metadatas','embeddings']: 문서 내용, 메타데이터, 벡터까지 모두 요청
results = collection.get(include=['documents','metadatas','embeddings'])
for doc, meta, id, emb in zip(results['documents'], results['metadatas'], results['ids'], results['embeddings']):
    print(f"id : {id}")
    print(f"doc : {doc}")
    print(f"metadata : {meta}")
    print(f"embedding : {emb[:5]}") # 384차원 벡터의 앞 5개 값 출력
    print(f"embedding dim : {len(emb)}")    # 벡터의 차원 수 (384) 확인
    print("--------------------")

id : doc1
doc : 문서1:금요일 퇴근 후에 헬스장
metadata : {'tag': 'mes1'}
embedding : [-0.00911812 -0.02242651  0.03233086 -0.0236039   0.02917333]
embedding dim : 384
--------------------
id : doc2
doc : 문서2:잠은 언제 자나~
metadata : {'tag': 'mes2'}
embedding : [ 0.00403785  0.05862309  0.06415996 -0.04665814  0.03444706]
embedding dim : 384
--------------------


In [ ]:
# 6. 자료 수정 (update - SQL의 UPDATE)
collection.update(
    ids=["doc2"],   # 수정할 문서의 ID 지정 (WHERE id = 'doc2')
    documents=['문서2:메세지 내용이 수정됨'],   # 문서 내용 변경. 이 새 내용으로 벡터가 재-생성됨.
    metadatas=[{'tag': 'edited-mes'}]       # 메타데이터 변경
)   

# 수정 후 다시 조회하여 변경 사항 확인 (doc2의 내용과 벡터가 바뀌었는지 확인)
results = collection.get(include=['documents','metadatas','embeddings'])
for doc, meta, id, emb in zip(results['documents'], results['metadatas'], results['ids'], results['embeddings']):
    print(f"id : {id}")
    print(f"doc : {doc}")
    print(f"metadata : {meta}")
    print(f"embedding : {emb[:5]}")
    print(f"embedding dim : {len(emb)}")
    print("--------------------")

id : doc1
doc : 문서1:금요일 퇴근 후에 헬스장
metadata : {'tag': 'mes1'}
embedding : [-0.00911812 -0.02242651  0.03233086 -0.0236039   0.02917333]
embedding dim : 384
--------------------
id : doc2
doc : 문서2:메세지 내용이 수정됨
metadata : {'tag': 'edited-mes'}
embedding : [ 0.02002782  0.01221266  0.06960154 -0.01440814  0.04374016]
embedding dim : 384
--------------------


In [ ]:
# 7. 자료 삭제 (delete - SQL의 DELETE)
collection.delete(ids=['doc1']) # ID를 기준으로 삭제 (DELETE FROM test WHERE id = 'doc1')
collection.delete(where={'tag':'edited-mes'})   # 메타데이터의 'tag'를 기준으로 삭제 (DELETE FROM test WHERE tag = 'edited-mes')

# 삭제 후 최종 조회 (결과가 비어 있어야 정상)
results = collection.get(include=['documents','metadatas','embeddings'])
for doc, meta, id, emb in zip(results['documents'], results['metadatas'], results['ids'], results['embeddings']):   # 이 부분은 문서가 없으므로 출력되지 않아야 함.
    print(f"id : {id}")
    print(f"doc : {doc}")
    print(f"metadata : {meta}")
    print(f"embedding : {emb[:5]}")
    print(f"embedding dim : {len(emb)}")
    print("--------------------")

추가 개념 정리: 벡터 DB의 데이터 관리

1. SentenceTransformerEmbeddingFunction의 역할이전에 all-MiniLM-L6-v2 모델이 384차원 벡터를 생성. 이 클래스를 사용함으로써 ChromaDB에 외부의 트랜스포머 모델을 연결하고, 모든 add 또는 update 작업 시 이 모델이 일관된 규칙으로 텍스트를 벡터로 변환하도록 강제
2. PersistentClient vs. chromadb.Client()
   구분         PersistentClient(path='./chroma_db')                                                         chromadb.Client()
   저장 방식        영구 저장 (Disk)                                                                             인메모리 (RAM)
   데이터 유지      코드를 종료해도 .chroma_db 폴더에 파일로 남아있어 데이터가 유지됨.          코드가 종료되거나 프로그램이 재시작되면 모든 데이터가 삭제됨.
   사용처           RAG 시스템 구축, 서비스 배포 등 데이터 유지가 필요한 경우                   간단한 테스트, 일회성 실습 등 데이터 유지가 필요 없는 경우
   
3. update() 시 벡터의 재생성 원리
   collection.update()를 사용할 때 documents 내용을 수정하면, ChromaDB는 해당 내용을 자동으로 embedding_fn에 다시 넣고 새로운 벡터를 생성하여 저장.
   중요성: 벡터는 텍스트의 의미. 텍스트가 바뀌면 의미도 바뀌므로, 반드시 새로운 벡터를 생성하여 데이터의 일관성을 유지해야 함. 만약 텍스트는 수정하고 벡터는 그대로 둔다면, 검색 시 잘못된 결과가 나옴.
   
4. delete(where={'필드':'값'})의 활용 where 조건을 이용한 삭제는 RDBMS의 DELETE FROM table WHERE 조건과 같습니다. 이를 통해 메타데이터(metadata)를 기준으로 유연하게 데이터를 삭제가능.
   예시: {'tag': 'edited-mes'}와 같이 메타데이터 필드를 활용하여 특정 조건에 맞는 문서를 쉽게 관리하고 제거 가능.


** 메타데이터는 검색 대상을 좁히는 도구이며, 벡터는 검색 대상을 의미적으로 찾는 도구